In [1]:
import numpy as np
import pandas as pd
import utm
import math
import os
import sys
sys.path.append('./src')
import matplotlib
import matplotlib.pyplot as plt
import display_osm
import display_gm
import plot
%matplotlib inline

# Load Data(2g&Jiading)

In [3]:
from data_cleaning.count import *

# 数据预处理
### 数据范围
* key_form GSM or LTE
* bounding_box 

### 数据错误
* (Mcc, Mnc)
    - (460, 0) 移动(程序初始化问题)
    - (460, 1) 联通

### 数据乱序
* Timestamp 乱序

### 数据重复
* First Tower 和第一个 Connected Tower重复
* 完全重复的数据

In [2]:
TimeStamp = ['MRTime']
Location = ['Longitude', 'Latitude', 'Altitude', 'Accuracy', 'Speed']
SignalRecord = ['Dbm', 'isGsm', 'SignalToNoiseRatio', 'EvdoEcio', 'Level']
Battery = ['BLevel', 'BCapacity']
Geomagnetism = ['X', 'Y', 'Z', 'X_theta', 'Y_theta', 'Z_theta']
Towers = ['Mcc', 'Mnc', 'RNCID', 'CellID', 'Arfcn', 'Basic_psc_pci', 'Lon', 'Lat', 'AsuLevel', 'SignalLevel', 'Dbm', 'Type']
Others = ['TrajID', 'IMEI', 'IMSI']
Basic_prop_len = len(TimeStamp) + len(Location) + len(SignalRecord) + len(Battery) + len(Geomagnetism)
NumConnLoc = Basic_prop_len + len(Towers)
len(Towers), Basic_prop_len

(12, 19)

In [3]:
def get_properties(num_tower):
    basic_prop = []
    basic_prop.extend(Others)
    basic_prop.extend(TimeStamp)
    basic_prop.extend(Location)
    basic_prop.extend(SignalRecord)
    basic_prop.extend(Battery)
    basic_prop.extend(Geomagnetism)
    basic_prop.append('Num_Connected')
    for i in range(num_tower):
        tower_i = [x + '_' + str(i+1) for x in Towers]
        basic_prop.extend(tower_i)
    return basic_prop

def load_data(folder, bounding_box, key_form, min_num=20):
    if not os.path.exists(folder):
        print 'not exist'
        return
    iterator = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    iterator = sorted(iterator, key=lambda x: int(x.split('.')[0]))
    data = []
    to_del = []
    for f in iterator:
        coors = load_file(os.path.join(folder, f), bounding_box, key_form, min_num=min_num)
        if type(coors) is str:
            print 'discard filename:', f, ' for reason:', coors
            to_del.append(os.path.join(folder, f))
            continue
        data.append(coors)
    for filename in to_del:
        os.remove(filename)
#     return pd.concat(data, axis=0)[get_properties(7)]
    return data

def outside(x, y, bounding_box):
    x0, x1, y0, y1 = bounding_box
    return x < x0 or x > x1 or y < y0 or y > y1
    
def load_file(filename, bounding_box, key_form, min_num=20):
    coors = pd.DataFrame()
    tr_id = int(filename.split('/')[-1].split('.')[0])
    with open(filename) as fp:
        data = fp.readlines()
        imei = int(data[0].strip().split(':')[-1])
        imsi = int(data[1].strip().split(':')[-1])
        total = int(data[7].strip())
        max_num_tower = 0
        result = []
        if key_form not in data[8]:
            return 'not' + key_form
        idx = 8
        last_stamp = -1
        for line in data[8:8+total]:
            mr = line.strip().split(',')
            lat, lng = float(mr[2]), float(mr[1])
            x, y, _, _ = utm.from_latlon(lat, lng)
            stamp = int(mr[0])
            if stamp == last_stamp:
                print 'duplicate'
            if outside(x, y, bounding_box):
                return 'outside'
#             print len(mr[Basic_prop_len:])
            assert (len(mr) - Basic_prop_len - 1) % len(Towers) == 0
            num_tower = int(mr[NumConnLoc])
#             print num_tower
#             print mr[NumConnLoc + len(Towers) -1]
            mr = mr[:Basic_prop_len] + mr[NumConnLoc:]
            max_num_tower = max(max_num_tower, num_tower)
            mr.insert(0, imsi)
            mr.insert(0, imei)
            mr.insert(0, tr_id)
            tup = tuple(mr)
#             if idx <= 10:
#                 print mr
            result.append(tup)
            idx += 1
            last_stamp = stamp
        properties = get_properties(max_num_tower)
        if len(result) > 0:
            coors = pd.DataFrame(result, columns=properties).sort_values(by=['MRTime']).reset_index(drop=True)
    return coors

In [4]:
jiading_folder = './data/jiading/'
disp_folder = './display'
bounding_box = (328000, 332000, 3461000, 3464000)
key_form = 'GSM'

In [22]:
fname = jiading_folder + 'uploads/' +  '60.txt'
coors = load_file(fname, bounding_box, key_form, min_num=20)

In [23]:
# coors[['MRTime', 'Dbm_1', 'Dbm_2', 'Dbm_3', 'Dbm_4', 'Dbm_5', 'Dbm_6', 'Dbm_7']]
# pd.DataFrame(coors[:10], columns=get_properties(7))
coors

,TrajID,IMEI,IMSI,MRTime,Longitude,Latitude,Altitude,Accuracy,Speed,Dbm,...,RNCID_7,CellID_7,Arfcn_7,Basic_psc_pci_7,Lon_7,Lat_7,AsuLevel_7,SignalLevel_7,Dbm_7,Type_7
0,60,862305032225518,460091042103139,1524055625523,121.20927625,31.28417336,14.0,6.0,0.27073973,-61,...,None,None,None,None,None,None,None,None,None,None
1,60,862305032225518,460091042103139,1524055628753,121.20927625,31.28417336,14.0,6.0,0.27073973,-59,...,None,None,None,None,None,None,None,None,None,None
2,60,862305032225518,460091042103139,1524055631680,121.2091642,31.28390897,62.0,18.0,6.713047,-65,...,None,None,None,None,None,None,None,None,None,None
3,60,862305032225518,460091042103139,1524055634674,121.20902792,31.28392255,91.0,7.0,3.6809916,-65,...,None,None,None,None,None,None,None,None,None,None
4,60,862305032225518,460091042103139,1524055637869,121.20913872,31.28413393,45.0,15.0,2.0831227,-65,...,None,None,None,None,None,None,None,None,None,None
5,60,862305032225518,460091042103139,1524055640825,121.20930197,31.28416657,53.0,5.0,4.324743,-65,...,None,None,None,None,None,None,None,None,None,None
6,60,862305032225518,460091042103139,1524055644049,121.20928254,31.2842816,19.0,8.0,2.9076624,-61,...,None,None,None,None,None,None,None,None,None,None
7,60,862305032225518,460091042103139,1524055646986,121.20917781,31.28435575,0.0,5.0,4.359002,-57,...,None,None,None,None,None,None,None,None,None,None
8,60,862305032225518,460091042103139,1524055650218,121.20917781,31.28435575,0.0,5.0,4.359002,-63,...,None,None,None,None,None,None,None,None,None,None
9,60,862305032225518,460091042103139,1524055653168,121.20935696,31.28457756,34.0,12.0,4.239257,-63,...,None,None,None,None,None,None,None,None,None,None


In [7]:
points = load_data(jiading_folder + 'raw/', bounding_box, key_form, min_num=20)

# Display

In [8]:
display_osm = reload(display_osm)
for traj in points:
#     print traj['TrajID'][0], len(traj)
    traj_id = int(traj['TrajID'][0])
    display_osm.df_to_html(traj, './display/raw/%d.html' % traj_id, more_info=True)

In [12]:
for traj in points:
    print traj['TrajID'][0], len(traj)

0 25
1 158
2 78
3 34
4 45
5 50
6 136
7 113
8 374
9 342
10 421
11 296
12 423
13 1033
14 843
15 116
16 865
17 247
18 348
19 336
20 192
21 260
22 235
23 194
24 430
25 191
26 304
27 424
28 530
29 85
30 92
31 61
32 147
33 245
34 80
35 109
36 120
37 90
38 114
39 88
40 181
41 54
42 98
43 136
44 76
45 54
46 76
47 76
48 54
49 239
50 322
51 113
52 76
53 52
54 75
55 55
56 51
57 58
58 49
59 107
60 86
61 60
62 61
63 85
64 485
65 62
66 89
67 90
68 81
69 65
70 85
71 50
72 76


In [9]:
data_2g = pd.concat(points, axis=0)[get_properties(7)]
data_2g.to_csv('./data/jiading/data_2g.csv', index=False)

In [10]:
plot = reload(plot)
fig_width=20
axis = bounding_box
for traj in points:
    trajs = []
    tr_id = int(traj['TrajID'][0])
    for i in range(len(traj)):
        point = traj.iloc[i]
        lat, lng = float(point['Latitude']), float(point['Longitude'])
        x, y, _, _ = utm.from_latlon(lat, lng)
        trajs.append((x, y))
    plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
    ca = plt.gca()
    plot.draw_traj_on_cells(trajs, ca, axis, 'r')
    plt.axis(axis)
    plt.title('traj id = ' + str(tr_id))
    plt.savefig('./display/raw/%d.png' % tr_id)
    plt.close()

# Rename file

In [5]:
def renamebytime(folder):
    if not os.path.exists(folder):
        print 'not exist'
        return
    iterator = (f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)))
    timeline = []
    for f in iterator:
#         if '4g' in f or '4G' in f:
#             print 'discard:', f
#             continue
#         record_time = int(f.strip().split('_')[-2])
#         timeline.append((f, record_time))
        record_time = int(f.strip().split('.')[0])
        timeline.append((f, record_time))
    timeline = sorted(timeline, key=lambda x: x[1])
    idx = 0
    for filename, _ in timeline:
        old_name = os.path.join(folder, filename)
        new_name = os.path.join(folder, '%d.txt' % idx)
        os.rename(old_name, new_name)
        idx += 1

In [6]:
renamebytime(jiading_folder + 'raw/')